In [1]:
import ee
import json
from osgeo import gdal
import geemap
import os
import json

import time
import subprocess as subp

from osgeo import gdal, gdal_array
from datetime import datetime, timedelta

ee.Initialize() # Initialize

In [2]:
year = 2017

In [5]:
f=  open('../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson')
data =json.load(f)

data = data['features'][0]['geometry']['coordinates'][0]
roi = ee.Geometry.Polygon(data)

features = geemap.fishnet(roi, rows=2, cols=2)
#features = ee.FeatureCollection([roi])

MyImage  = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS") \
                    .filterBounds(roi).filterDate(str(year)+'-01-01',str(year+1)+'-01-01') \
                    .mean().clip(roi)

In [6]:
geemap.download_ee_image_tiles(
    MyImage, features, "LULC_"+str(year), prefix="LULC_", crs="EPSG:4326", scale=10,
)

#This step shows raw file size in GB but not worry, actual downloaded file will be in less than 100 mb

Consider adjusting `region`, `scale` and/or `dtype` to reduce the LULC_1.tif download size (raw: 4.12 GB).


LULC_1.tif: |          | 0.00/4.12G (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


Consider adjusting `region`, `scale` and/or `dtype` to reduce the LULC_2.tif download size (raw: 4.12 GB).


LULC_2.tif: |          | 0.00/4.12G (raw) [  0.0%] in 00:00 (eta:     ?)

...

LULC_3.tif: |          | 0.00/4.12G (raw) [  0.0%] in 00:00 (eta:     ?)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the LULC_4.tif download size (raw: 4.12 GB).


LULC_4.tif: |          | 0.00/4.12G (raw) [  0.0%] in 00:00 (eta:     ?)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the LULC_5.tif download size (raw: 4.12 GB).


LULC_5.tif: |          | 0.00/4.12G (raw) [  0.0%] in 00:00 (eta:     ?)

In [16]:
'''
# Export the image, specifying scale and region.

task = ee.batch.Export.image.toDrive(**{
    'image': MyImage,
    'description': 'LULC_'+str(year),
    'folder':'LULC_GEE',
    'scale': 10,
    'region': table.getInfo()['coordinates'],
    'maxPixels': 1e13
})
task.start()

import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

'''

"\n# Export the image, specifying scale and region.\n\ntask = ee.batch.Export.image.toDrive(**{\n    'image': MyImage,\n    'description': 'LULC_'+str(year),\n    'folder':'LULC_GEE',\n    'scale': 10,\n    'region': table.getInfo()['coordinates'],\n    'maxPixels': 1e13\n})\ntask.start()\n\nimport time \nwhile task.active():\n  print('Polling for task (id: {}).'.format(task.id))\n  time.sleep(5)\n\n"

In [4]:
basepath='LULC_'+str(year)+'/'
arr = os.listdir(basepath)

cmd= "gdal_merge.py -of GTiff -co TILED=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=2 -o LULC_"+str(year)+".tif"
for i in arr:
    #cmd="gdal_translate clipped/"+str(i)+" LULC-"+str(year)+"/"+str(i)+" -co COMPRESS=LZW -co ZLEVEL=9 -co TILED=YES"
    cmd = cmd+" "+basepath+i
print(cmd)
try:
    subp.check_call(str(cmd), shell=True)
except:
    print("end")
time.sleep(1)

cmd="gdalwarp -dstnodata -9999 -cutline TL_state_shapefile_for_clip.geojson -crop_to_cutline merged.tif LULC_"+str(year)+".tif -co TILED=YES -co COMPRESS=DEFLATE -co ZLEVEL=9 -co PREDICTOR=2"
print(cmd)
try:
    subp.check_call(str(cmd), shell=True)
except:
    print("end")
time.sleep(1)
'''
cmd="rm -r LULC_"+str(year)+" merged.tif"
print(cmd)
try:
    subp.check_call(str(cmd), shell=True)
except:
    print("end")
'''

gdal_merge.py -of GTiff -co BIGTIFF=YES -co PREDICTOR=2 -co TILED=YES -co ZLEVEL=9 -o LULC_2017.tif LULC_2017/LULC_1.tif LULC_2017/LULC_2.tif LULC_2017/LULC_5.tif LULC_2017/LULC_3.tif LULC_2017/LULC_4.tif
0

Above code wrapped in a function

In [ ]:
def download_lulc(region, year):

    f=  open('../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson')
    data =json.load(f)

    data = data['features'][0]['geometry']['coordinates'][0]
    roi = ee.Geometry.Polygon(data)

    features = geemap.fishnet(roi, rows=2, cols=2)
    #features = ee.FeatureCollection([roi])

    MyImage  = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS") \
                        .filterBounds(roi).filterDate(str(year)+'-01-01',str(year+1)+'-01-01') \ 
                        

    geemap.download_ee_image_tiles(
        MyImage, features, "LULC_"+str(year), prefix="LULC_", crs="EPSG:4326", scale=10,  #folder naming is optinal
    )

#This step shows raw file size in GB but not worry, actual downloaded file will be in less than 100 mb

    basepath='LULC_'+str(year)+'/'
    arr = os.listdir(basepath)

    cmd= "gdal_merge.py -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 -o LULC_"+str(year)+".tif"
    for i in arr:
        #cmd="gdal_translate clipped/"+str(i)+" LULC-"+str(year)+"/"+str(i)+" -co COMPRESS=LZW -co ZLEVEL=9 -co TILED=YES"
        cmd = cmd+" "+basepath+i
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

    cmd="gdalwarp -dstnodata -9999 -cutline TL_state_shapefile_for_clip.geojson -crop_to_cutline merged.tif LULC_"+str(year)+".tif"
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)
    cmd="rm -r LULC_"+str(year)+" merged.tif"
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
